In [1]:
from data_extraction import fetch_api_response
from data_processing import get_rating_with_std, get_rating_fixed
from data_analysis import plot_data
import json

response = fetch_api_response()
overview = response.json()['fear_and_greed']
print(json.dumps(overview, indent=3))

{
   "score": 32.5176470588235,
   "rating": "fear",
   "timestamp": "2025-10-10T18:47:38+00:00",
   "previous_close": 49.0571428571429,
   "previous_1_week": 53.6857142857143,
   "previous_1_month": 49.8571428571429,
   "previous_1_year": 70.28571428571429
}


In [2]:
import pandas as pd

df = pd.DataFrame(overview, index=[0])
df

,score,rating,timestamp,previous_close,previous_1_week,previous_1_month,previous_1_year
0,32.517647,fear,2025-10-10T18:47:38+00:00,49.057143,53.685714,49.857143,70.285714


In [3]:
dfs = []

titles = {
    'fear_and_greed_historical': 'Fear & Greed Index Over Time',
    'market_momentum_sp500': 'S&P 500 Market Momentum',
    'market_momentum_sp125': 'S&P 125 Market Momentum',
    'stock_price_strength': 'Stock Price Strength',
    'stock_price_breadth': 'Stock Price Breadth',
    'put_call_options': 'Put/Call Options Ratio',
    'market_volatility_vix': 'Market Volatility (VIX)',
    'market_volatility_vix_50': 'VIX 50-day Moving Average',
    'junk_bond_demand': 'Junk Bond Demand',
    'safe_haven_demand': 'Safe Haven Demand'
}


for key in titles:
    data = response.json()[key]['data']
    df_aux = pd.DataFrame(data)[['x', 'y']]
    df_aux.rename(columns={'y': key}, inplace=True)
    df_aux['date'] = pd.to_datetime(df_aux['x'], unit='ms')
    df_aux.drop(columns=['x'], inplace=True)
    df_aux.set_index('date', inplace=True)
    dfs.append(df_aux)
df = pd.concat(dfs, axis=1)
df.sort_index(inplace=True)

In [4]:
#calculations
df['sp500_momentum'] = df['market_momentum_sp500'] - df['market_momentum_sp125']
df['vix_momentum'] = df['market_volatility_vix'] - df['market_volatility_vix_50']

calculation_titles = {
    # 'fear_and_greed_historical': ['Fear & Greed Index Over Time', False], regra mais simples
    'sp500_momentum': ['S&P 500 Market Momentum', False],
    'vix_momentum': ['Market Volatility (VIX) Minus 50-day MA', True],
    'safe_haven_demand': ['Safe Haven Demand', True],
    'junk_bond_demand': ['Junk Bond Demand', True],
    'put_call_options': ['Put/Call Options Ratio', True]
}

for key, title_and_is_inverted in calculation_titles.items():
    title = title_and_is_inverted[0]
    is_inverted = title_and_is_inverted[1]
    mean = df[key].mean()
    std = df[key].std()
    df[f'{key}_rating'] = df[key].apply(lambda v: get_rating_with_std(v, mean, std, inverted=is_inverted))
df['fear_and_greed_historical_rating'] = df['fear_and_greed_historical'].apply(lambda v: get_rating_fixed(v))

In [5]:
#plot
graph_data = [
    # 'fear_and_greed_historical': ['Fear & Greed Index Over Time', False], regra mais simples
    ['fear_and_greed_historical', 'Historical Fear & Greed Index', 'fear_and_greed_historical_rating', None, None],
    ['market_momentum_sp500', 'S&P 500 Market Momentum', 'sp500_momentum_rating', 'market_momentum_sp125', 'SP500 125 MA'],
    ['market_volatility_vix', 'Market Volatility (VIX) Minus 50-day MA', 'vix_momentum_rating', 'market_volatility_vix_50', 'VIX 50 MA'],
    ['safe_haven_demand', 'Safe Haven Demand', 'safe_haven_demand_rating', None, None],
    ['junk_bond_demand', 'Junk Bond Demand', 'junk_bond_demand_rating', None, None],
    ['put_call_options', 'Put/Call Options Ratio', 'put_call_options_rating', None, None]
]

for data in graph_data:
    plot_data(df, key=data[0], title=data[1], rating_key=data[2], complementary_key=data[3], title_comp_key=data[4])